In [ ]:
import os
import glob
import warnings
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


from copy import deepcopy
from tqdm.auto import tqdm

from multiresticodm.utils import *
from multiresticodm.config import Config
from multiresticodm.outputs import OutputSummary

import multiresticodm.spatial_interaction_model as SIM_Model

In [ ]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

## Import samples

In [ ]:
# Specify experiment id
experiment_id = "SIM_NN_SweepedNoise_16_05_2023_20_09_04"
experiment_group_id = 'exp1/'

In [ ]:
# Create new logging object
logger = setup_logger(
    __name__,
    console_level = 'INFO',
    file_level = 'EMPTY'
)

In [ ]:
# Define path to outputs
outputs_dir = f"../data/outputs"
outputs_path = os.path.join(
    outputs_dir,
    f"cambridge_work_commuter_lsoas_to_msoas/{experiment_group_id}{experiment_id}"
)
# Read config
conf = Config(
    path = os.path.join(outputs_path,"config.json"),
    logger = logger
)

In [ ]:
# Output processing settings
output_settings = dict(
    logging_mode = 'INFO',
    out_directory = outputs_dir,
    dataset_name = [f"cambridge_work_commuter_lsoas_to_msoas/{experiment_group_id}"],
    # experiment_type = ['SIM_NN'],
    directories = ['SIM_NN_SweepedNoise_16_05_2023_20_09_04'],
    metadata_keys = ['sigma','name'],
    sample = ['intensity','total_loss'],
    statistic = [('','mean&','iter&')],
    burnin_thinning_trimming = {'iter':{"burnin":0,"thinning":1,"trimming":None}},
    force_reload = True
)

# Compile outputs summary
outsum = OutputSummary(
    settings = output_settings,
    logger = logger
)

In [ ]:
# Gather outputs
outputs = outsum.get_folder_outputs(0,outsum.output_folders[0])

In [ ]:
index = 4

In [ ]:
outputs.data.alpha[index].coords

In [ ]:
np.round(outputs.data.alpha[index][0,0,0,0,0,:10].values,4)

In [ ]:
np.round(outputs.data.beta[index][0,0,0,0,0,:10].values,4)

In [ ]:
np.round(np.exp(outputs.data.log_destination_attraction[index][0,0,0,0,0,:10].values),4)

In [ ]:
outputs.data.total_loss[index][0,0,0,0,0,:10].values